In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11135,2024-05-23,República Tcheca Nbl,12:30,Ostrava,Decin,2.80,1.40,156.5,1.87,1.79,0.0,0.00,0.00,IgWzVHO6,0.357143,0.714286,0.534759,0.558659,0.071429,608.260,120.528806,0.198153,0.0,0.000000,NaN,395.90,9.410,2.355419,0.250310,-131.0,152.008,52.452437,0.345064,1.2,1.643168,1.369306,88.14,1.916,0.699628,0.365151,-8.0,74,78,5.35,1.13,361.076,223.634,0.471405,0.030912,NaN,6.400000e-01,1.280000e-01,1.406250e+01,0.569890,0.6,0.030110,0.45,0.090,4.444444,0.446677,0.5,0.053323
11136,2024-05-23,Israel Liga Leumit,14:00,Elitzor Netania,Ironi Nahariya,1.36,2.97,161.5,1.81,1.89,0.0,0.00,0.00,xOtcUcQF,0.735294,0.336700,0.552486,0.529101,0.071994,139.700,49.238775,0.352461,3.0,0.000000,0.000000,225.04,1.608,0.422102,0.262501,61.0,150.664,33.523375,0.222504,1.8,1.643168,0.912871,120.12,1.846,0.434200,0.235211,-13.0,97,77,2.32,1.56,112.512,174.244,0.525839,0.030578,NaN,1.590000e+00,3.180000e-01,1.132075e+00,0.707615,0.9,0.192385,3.07,0.614,3.208469,0.520748,0.6,0.079252
11137,2024-05-23,Itália Série A2,15:30,Verona,Trapani,2.62,1.45,153.5,1.85,1.85,0.0,0.00,0.00,dxkVHhvn,0.381679,0.689655,0.540541,0.540541,0.071335,157.794,73.286190,0.464442,2.4,1.341641,0.559017,282.58,2.048,1.112147,0.543040,34.0,102.688,15.281593,0.148816,2.4,1.341641,0.559017,95.16,1.254,0.083845,0.066862,51.0,71,78,3.98,1.22,108.508,106.114,0.406543,0.000000,NaN,-1.160000e+00,-2.320000e-01,-6.982759e+00,0.755958,0.8,0.044042,-1.08,-0.216,-2.083333,0.778867,0.7,-0.078867
11138,2024-05-23,Kosovo Superliga,14:00,Trepca,Golden Eagle Ylli,1.45,2.62,156.5,1.80,1.86,0.0,0.00,0.00,W2soaW7k,0.689655,0.381679,0.555556,0.537634,0.071335,115.360,15.902196,0.137848,2.4,1.341641,0.559017,119.85,1.434,0.216402,0.150908,31.0,168.768,35.589419,0.210878,1.2,1.643168,1.369306,215.28,2.150,0.455577,0.211896,-12.0,85,78,1.41,2.76,0.000,183.226,0.406543,0.023184,NaN,0.000000e+00,0.000000e+00,inf,0.000000,0.0,0.000000,-0.70,-0.140,-11.571429,0.000000,0.0,0.000000
11139,2024-05-23,Lituânia Lkl,12:50,Lietkabelis,Zalgiris Kaunas,3.97,1.23,162.5,1.88,1.85,0.0,0.00,0.00,jX7WFMhc,0.251889,0.813008,0.531915,0.540541,0.064897,208.862,127.436777,0.610148,1.8,1.643168,0.912871,428.40,2.546,1.933463,0.759412,-6.0,105.076,10.471107,0.099653,2.4,1.341641,0.559017,104.16,1.108,0.125379,0.113158,98.0,72,93,5.95,1.12,134.494,114.452,0.745182,0.011374,NaN,-2.720000e+00,-5.440000e-01,-5.459559e+00,0.724519,0.5,-0.224519,-1.46,-0.292,-0.787671,0.822109,0.7,-0.122109
11140,2024-05-23,Nova Zelândia Nbl,04:30,Whai,Canterbury Rams,6.36,1.09,169.5,1.81,1.85,0.0,0.00,0.00,dSw8w5Om,0.157233,0.917431,0.552486,0.540541,0.074664,369.772,234.071278,0.633015,0.6,1.341641,2.236068,426.30,5.074,3.173008,0.625346,-76.0,153.520,55.843232,0.363752,2.4,1.341641,0.559017,140.18,1.604,0.480240,0.299401,44.0,70,86,6.09,1.63,0.000,0.000,1.000390,0.015456,NaN,0.000000e+00,0.000000e+00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11141,2024-05-23,Paraguai Lnb,20:30,Sol De America,Dep. San Jose,12.00,22.00,152.5,1.85,1.85,0.0,0.00,0.00,MefTzgA6,0.083333,0.045455,0.540541,0.540541,-0.871212,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,611.556,1145.267509,1.872711,2.4,1.341641,0.559017,127.19,6.572,11.980838,1.823012,51.0,56,79,22.00,1.61,0.000,0.000,0.415945,0.000000,NaN,0.0000

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11156,12:40,Israel Superliga,Hapoel Beer Sheva,H. Afula,1.63,Back Home
11163,13:30,Alemanha Bbl,Vechta,Chemnitz,2.77,Back Home
